In [1]:
!pip install python-multipart

In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn

# Cargamos el modelo preentrenado

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import cv2

In [2]:
# Función para entrenar una red neuronal convolucional basada en el modelo ResNet

class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        # Cargar el modelo ResNet preentrenado
        self.resnet = models.resnet50(pretrained=True)

        # Reemplazar la capa final de clasificación de ResNet para adaptarse al número de clases deseado
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Remover la capa final existente

        # Definir las nuevas capas de clasificación
        self.fc1 = nn.Linear(in_features, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)  # Pasar por el ResNet sin la capa final
        x = self.fc1(x)    # Pasar por la capa densa 1
        x = self.relu(x)   # Aplicar ReLU
        x = self.fc2(x)    # Pasar por la capa densa 2
        x = self.softmax(x)  # Aplicar Softmax
        return x


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

model = ResNetModel().to(device)

# Ruta al archivo checkpoint
ruta_checkpoint = 'Modelo.pt'

# Cargar el estado del modelo desde el checkpoint
model.load_state_dict(torch.load(ruta_checkpoint, map_location=device))


c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5852\1379765419.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during u

<All keys matched successfully>

# Usamos FastAPI para desplegar el modelo en internet a través de una interfaz

## Imports y funciones necesarias para el despliegue

In [4]:
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.responses import RedirectResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
import cv2
import numpy as np
import io
from fastapi.responses import StreamingResponse
import base64

In [5]:
# Definir las transformaciones de preprocesamiento
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Cambia de BGR a RGB
    image = image.astype(np.float32) / 255.0  # Normaliza a [0, 1]
    tensor_image = torch.tensor(image).permute(2, 0, 1)  # Cambia la forma a CxHxW
    tensor_image = tensor_image.unsqueeze(0)  # Añade la dimensión del batch
    return tensor_image


def resize_image(img):
    # Convierte a un array NumPy desde el contenido de la imagen
    nparr = np.frombuffer(img, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  # Decodifica la imagen

    if image is None:
        raise ValueError("Error al decodificar la imagen.")

    # Verifica la forma de la imagen
    shape = image.shape

    # Redimensionar si no tiene el tamaño esperado
    if shape[0] == 224 and shape[1] == 224:
        return image
    else:
        dim = (224, 224)
        resized = cv2.resize(image, dim)
        return resized

def predict(image_tensor):
    with torch.no_grad():#desactivamos el calculo de gradientes
        output = model(image_tensor)#hacer la inferencia
        _, predicted = torch.max(output.data, 1)#obtener la clase predicha
    return predicted.item()

In [6]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Montar archivos estáticos (CSS, imágenes, etc.)
app.mount("/static/css", StaticFiles(directory = "templates/stylescss"), name="css")
app.mount("/static/images", StaticFiles(directory = "templates/images"), name="images")
app.mount("/static/js", StaticFiles(directory = "templates"), name="js")

# Configurar las plantillas
templates = Jinja2Templates(directory="templates/")


@app.get("/")
async def landing_page(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "title": "Agro Diagnóstico IA"})

@app.get("/upload/")
async def upload_page(request: Request):
    return templates.TemplateResponse("plataforma.html", {"request": request})


@app.post("/upload/")
async def handle_upload(request: Request, file: UploadFile = File(...)):
    try:
        content = await file.read()  # Lee el contenido del archivo
        nparr = np.frombuffer(content, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if image is None:
            raise ValueError("Error al decodificar la imagen.")


        # Redimensionamos la imagen
        resized_image = cv2.resize(image, (224, 224))  # Cambia a image
        preprocess = transforms.Compose([
            transforms.ToTensor(),  # Convert the image to a PyTorch tensor
            ])
        image_tensor = preprocess(resized_image)
        image_tensor = image_tensor.unsqueeze(0).to(device)  # Agregar dimensión de lote y mover a dispositivo

        # Preparamos la imagen para el modelo
        #tensor_image = preprocess_image(resized_image)

        with torch.no_grad():
            model.eval()
            output = model(image_tensor)  # Obtener la salida del modelo
            print(f"Salida del modelo: {output}")  # Verifica la salida del modelo

        # Obtener la clase predicha
        class_names = ["Cordana", "Saludable", "Pestalotiopsis", "Sigatoka"]
        _, prediction = torch.max(output, 1)  # Obtener la clase con mayor probabilidad
        predicted_index = prediction.item()
        predicted_class_name = class_names[predicted_index]  # Obtener el nombre de la clase

        # Codificamos la imagen redimensionada para la respuesta
        _, buffer = cv2.imencode('.jpg', resized_image)  # Codifica la imagen redimensionada en JPEG
        byte_io = io.BytesIO(buffer)  # Crea objeto BytesIO para la imagen
        return StreamingResponse(byte_io, media_type="image/jpeg", headers={"X-Predicted-Class": predicted_class_name})  # Devuelve la imagen procesada

    except Exception as e:
        print(f"Error: {e}")  # Muestra el error en la consola
        return {"error": str(e)}  # Devuelve el error como respuesta JSON



## Despliegue de la aplicación

Ejecuta la siguiente celda para desplegar el modelo online y cargar las imágenes:
Es necesario que todas las anteriores celdas hayan sido previamente ejecutadas.

In [7]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2mWWl9c2JDShbjp2DF10HlzqYPy_4QUs4RZMwQJoERjDM5HHp"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)

INFO:     Started server process [5852]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://9c7e-177-254-18-219.ngrok-free.app
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET / HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/css/home.css HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/logoAgroDiagnosticoIA.png HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/dos.PNG HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/imagendron1.png HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/tres.PNG HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/uno.PNG HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/imageninicio.PNG HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "GET /static/images/iconADIA.ico HTTP/1.1" 200 OK
INFO:     2800:e2:1a00:8dad:b518:66f7:a4c2:4775:0 - "G

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5852]


# PRUEBA DEL MODELO CARGADO LOCALMENTE SIN TENER EN CUENTA FAST API

In [48]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

PATH = 'Modelo.pt'
model1 = ResNetModel().to(device)
model1.load_state_dict(torch.load(PATH, map_location=device))
model1.eval()

path = 'Imagenes_prueba/Test2.jpg' # Ruta imagen a predecir

im = cv2.imread(path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
result = resized_image = cv2.resize(im, (224, 224))

preprocess = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

img_tensor = preprocess(result)
img_tensor = img_tensor.unsqueeze(0) # Add a batch dimension
img_tensor = img_tensor.to(device)

with torch.no_grad():  # No need to compute gradients
    output = model1(img_tensor)

# The output will be raw scores (logits); apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Get the predicted class index
_, predicted_class = torch.max(output, 1)

# plt.imshow(result)
print(predicted_class)

Using cpu device
tensor([2])


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11068\3309231191.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(PATH, map_location=devi